In [1]:
!pip install z3-solver
from z3 import *
import random as r

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53.8 MB 225 bytes/s 


In [3]:
n = 7
prob = 40

In [8]:
def declare(k):
    state = {}
    for i in range(n+1):
        for j in range(n+1):
            state['a%s%s' % (i,j)] = Const('a%s%s' % (i,j), BoolSort())
    return state

def init(state):

    c = (r.randint(4,n-3), r.randint(4,n-3))
   
    #center cells
    center = And(state['a%s%s'%(c[0]-1,c[1])] == True, 
        state['a%s%s'%(c[0]-1,c[1]+1)] == True, 
        state['a%s%s'%(c[0]-1,c[1]-1)] == True,
        state['a%s%s'%c] == True, 
        state['a%s%s'%(c[0],c[1]+1)] == True,
        state['a%s%s'%(c[0],c[1]-1)] == True,
        state['a%s%s'%(c[0]+1,c[1])] == True, 
        state['a%s%s'%(c[0]+1,c[1]+1)] == True, 
        state['a%s%s'%(c[0]+1,c[1]-1)] == True)

    #cells other than center and border
    cells = [state['a%s%s'%(i,j)] for i in range(1, n+1) for j in range(1, n+1) if (i,j) not in [(c[0]-1,c[1]),
                                                                                                 (c[0]-1,c[1]+1),
                                                                                                 (c[0]-1,c[1]-1),
                                                                                                 (c[0],c[1]+1),
                                                                                                 (c[0],c[1]-1),
                                                                                                 (c[0]+1,c[1]),
                                                                                                 (c[0]+1,c[1]+1),
                                                                                                 (c[0]+1,c[1]-1),
                                                                                                 c]]

    #border cells
    border = [state['a0%s'%i] for i in range(1,n+1)]
    border.extend([state['a%s0'%i] for i in range(1, n+1)])

    odd = [True] * prob
    odd.extend([False] * (100-prob))

    return And(center, Sum(border) == sum(odd), Sum(cells) == 0)

def trans(c,p):
    born = And(Sum((c[0]-1,c[1]),(c[0]-1,c[1]+1),(c[0]-1,c[1]-1),(c[0],c[1]+1),(c[0],c[1]-1),(c[0]+1,c[1]),(c[0]+1,c[1]+1),(c[0]+1,c[1]-1),c) == 3,
               p == True)
               
    survive = And(Sum((c[0]-1,c[1]),(c[0]-1,c[1]+1),(c[0]-1,c[1]-1),(c[0],c[1]+1),(c[0],c[1]-1),(c[0]+1,c[1]),(c[0]+1,c[1]+1),(c[0]+1,c[1]-1),c) == 2 or 3, p == True)


    die = And(Or(Sum((c[0]-1,c[1]),(c[0]-1,c[1]+1),(c[0]-1,c[1]-1),(c[0],c[1]+1),(c[0],c[1]-1),(c[0]+1,c[1]),(c[0]+1,c[1]+1),(c[0]+1,c[1]-1),c) < 2,
                 Sum((c[0]-1,c[1]),(c[0]-1,c[1]+1),(c[0]-1,c[1]-1),(c[0],c[1]+1),(c[0],c[1]-1),(c[0]+1,c[1]),(c[0]+1,c[1]+1),(c[0]+1,c[1]-1),c) > 3),
               p == False)
    
    return Or([born, survive, die]) 

In [5]:
def gen_trace(declare,init,trans,k):

    s = Solver()
    trace = [declare(i) for i in range(k)]

    s.add(init(trace[0]))
    print(trace[0])

    for i in range(k-1):
        for cell in trace[i]:
            s.add(cell,trace[i+1])

    if s.check() == sat:
        m = s.model()
        print("model is sat")


In [6]:
gen_trace(declare,init,trans,20)

{'a00': a00, 'a01': a01, 'a02': a02, 'a03': a03, 'a04': a04, 'a05': a05, 'a06': a06, 'a07': a07, 'a10': a10, 'a11': a11, 'a12': a12, 'a13': a13, 'a14': a14, 'a15': a15, 'a16': a16, 'a17': a17, 'a20': a20, 'a21': a21, 'a22': a22, 'a23': a23, 'a24': a24, 'a25': a25, 'a26': a26, 'a27': a27, 'a30': a30, 'a31': a31, 'a32': a32, 'a33': a33, 'a34': a34, 'a35': a35, 'a36': a36, 'a37': a37, 'a40': a40, 'a41': a41, 'a42': a42, 'a43': a43, 'a44': a44, 'a45': a45, 'a46': a46, 'a47': a47, 'a50': a50, 'a51': a51, 'a52': a52, 'a53': a53, 'a54': a54, 'a55': a55, 'a56': a56, 'a57': a57, 'a60': a60, 'a61': a61, 'a62': a62, 'a63': a63, 'a64': a64, 'a65': a65, 'a66': a66, 'a67': a67, 'a70': a70, 'a71': a71, 'a72': a72, 'a73': a73, 'a74': a74, 'a75': a75, 'a76': a76, 'a77': a77}


Z3Exception: ignored